### This Jupyter notebook performs coordinate extraction and analysis from proton (ZP) and neutron (ZN) detector image data.

Key steps include:
1. **Data Loading**: Proton and neutron detector images are loaded from `.pkl` files. The files present already limited data according to the filtering on the minimal intensity of samples.
2. **Coordinate Extraction**: The pixel coordinates with the maximum values (representing peak signals) are computed for both ZP and ZN.
3. **Data Saving**: Extracted coordinates are saved for training and downstream analysis.
4. **Coordinate Analysis**:
   - Histograms of x and y coordinate distributions.
   - KDE heatmaps to show spatial density.
   - Basic statistical summary (min, max, mean, median, std).
   - Sample image visualization with overlaid coordinates.
   - Correlation analysis between image intensity and coordinate location.

External utility used: `get_max_value_image_coordinates()` from `utils.utils`


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from expertsim.train.utils import get_max_value_image_coordinates

# Proton data

`data_zp_path` should contain the path to the data pickle file containing the images from the proton calorimeter generated from `data_filtering.ipynb`.

In [3]:
data_zp_path = r"data/data_proton_photonsum_proton_1_2312.pkl"  # load the path to the proton data
data_zp = pd.read_pickle(data_zp_path)

In [4]:
positions = []
for img in data_zp:
    coordinates_max = get_max_value_image_coordinates(img)
    positions.append(coordinates_max)

In [5]:
coordinates_df = pd.DataFrame(data=positions, columns=('max_x', 'max_y'))

In [6]:
coordinates_df.head()

,max_x,max_y
0,1,9
1,11,1
2,37,16
3,17,40
4,29,43


In [18]:
# save the coordinates to a pickle file to be ready for training
coordinates_df.to_pickle('data/data_coord_photonsum_proton_1_2312.pkl')

# Neutron data

`data_zn_path` should contain the path to the data pickle file containing the images from the neutron calorimeter generated from `data_filtering.ipynb`.

In [3]:
data_zn_path = r"data\data_neutron_photonsum_neutron_1_3360.pkl"  # load the path to the neutron data
data_zn = pd.read_pickle(data_zn_path)

In [4]:
positions = []
for img in data_zn:
    coordinates_max = get_max_value_image_coordinates(img)
    positions.append(coordinates_max)

In [5]:
coordinates_df = pd.DataFrame(data=positions, columns=('max_x', 'max_y'))

In [11]:
coordinates_df.head()

,max_x,max_y
0,26,14
1,1,9
2,0,40
3,22,17
4,11,1


In [7]:
# save the coordinates to a pickle file to be ready for training
coordinates_df.to_pickle('data/data_coord_photonsum_neutron_1_3360.pkl')

# Analyze the coordinates

A. Dataset Analysis
Coordinate Distribution Analysis

1. Generate histograms of coordinate distributions (x and y separately)
2. Create heatmaps showing spatial density of coordinates
3. Calculate statistical measures (min, max, mean, median, std) for both coordinates
4. Visualize sample images with their corresponding coordinates

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# Global font size increase for better readability
plt.rcParams.update({'font.size': 14})

def coordinate_distribution_analysis(df, output_dir, image_shape=(56, 30)):
    """Save coordinate distribution plots and return stats"""
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))

    axs[0].hist(df['max_x'], bins=50, color='blue', alpha=0.8, label='max pixel value coordiante')
    axs[0].set_xlabel('X Coordinate')
    axs[0].set_ylabel('Frequency')
    axs[0].set_title('Distribution of X Coordinates')
    # axs[0].legend()

    axs[1].hist(df['max_y'], bins=50, color='green', alpha=0.8, label='max_y Distribution')
    axs[1].set_xlabel('Y Coordinate')
    axs[1].set_ylabel('Frequency')
    axs[1].set_title('Distribution of Y Coordinates')
    # axs[1].legend()

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'coordinate_histograms.png'))
    plt.close()

    # Calculate statistics
    stats = df[['max_x', 'max_y']].agg(['min', 'max', 'mean', 'median', 'std']).T
    print("Statistical summary:\n", stats)
    return stats

def visualize_sample_images(df, data, output_dir, n_samples=5, image_shape=(56, 30)):
    """Save sample images with coordinates using a colored palette"""
    sample_indices = np.random.choice(df.index, n_samples, replace=False)
    if image_shape[0] == 56:
        fig, axs = plt.subplots(1, n_samples, figsize=(2.25*n_samples + 0.75*n_samples, 5), sharey=True)  # Increased width for colorbars
    else:
        fig, axs = plt.subplots(1, n_samples, figsize=(2*n_samples + 2*n_samples, 5), sharey=True)

    img_height, img_width = data.shape[1:3] if len(data.shape) > 2 else image_shape

    for i, idx in enumerate(sample_indices):
        # Convert coordinates to integers
        x_int = int(round(df.loc[idx, 'max_x']))
        y_int = int(round(df.loc[idx, 'max_y']))

        # Compute per-image min and max for individual scaling
        img_min = np.min(data[idx])
        img_max = np.max(data[idx])

        # Plot image with inverted Y-axis (0 on top)
        im = axs[i].imshow(data[idx], cmap='viridis', extent=[0, img_width, img_height, 0],
                           vmin=img_min, vmax=img_max)  # Individual scaling and inverted Y

        axs[i].set_title(f"X: {x_int}\nY: {y_int}", fontsize=14)  # Use integer coordinates
        axs[i].grid(True, alpha=0.1)  # Add grids with very low opacity

        # Set grid every 5 values
        if image_shape[0] == 56:
            axs[i].set_xticks(np.arange(0, img_width + 1, 5))
            axs[i].set_yticks(np.arange(0, img_height + 1, 5))
        else:
            axs[i].set_xticks(np.arange(0, img_width + 1, 10))
            axs[i].set_yticks(np.arange(0, img_height + 1, 10))

        # Add axis labels (OY as "X coordinate", OX as "Y coordinate")
        if i == 0:  # Only label leftmost for shared axes
            axs[i].set_ylabel('X coordinate')  # OY (vertical) as "X coordinate"
        axs[i].set_xlabel('Y coordinate')  # OX (horizontal) as "Y coordinate"

        # Add individual colorbar for each subplot
        cbar = fig.colorbar(im, ax=axs[i], orientation='vertical', fraction=0.046, pad=0.04)
        cbar.set_label('Intensity', fontsize=12)

    plt.suptitle('Sample Images with Corresponding Coordinates', fontsize=16)
    plt.subplots_adjust(wspace=0.3, hspace=0.05, right=0.95)  # Increased wspace for colorbars

    plt.savefig(os.path.join(output_dir, 'sample_images.png'))
    plt.close()
    return sample_indices


def image_coordinate_relationship(df, data, sample_indices, output_dir):
    """Save correlation analysis plots with improved labels"""
    image_features = data[sample_indices].mean(axis=(1, 2))
    coords_x = df.loc[sample_indices, 'max_x']
    coords_y = df.loc[sample_indices, 'max_y']

    # Calculate correlations
    corr_x = np.corrcoef(image_features, coords_x)[0, 1]
    corr_y = np.corrcoef(image_features, coords_y)[0, 1]
    print(f"Correlation between mean image intensity and max_x: {corr_x:.3f}")
    print(f"Correlation between mean image intensity and max_y: {corr_y:.3f}")

    # Create scatter plots with improved labels and colors
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.scatter(coords_x, image_features, color='blue', alpha=0.6, label='Data Points')
    plt.xlabel('Maximum X Coordinate')
    plt.ylabel('Mean Pixel Intensity')
    plt.title('Mean Intensity vs Maximum X Coordinate')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.scatter(coords_y, image_features, color='green', alpha=0.6, label='Data Points')
    plt.xlabel('Maximum Y Coordinate')
    plt.ylabel('Mean Pixel Intensity')
    plt.title('Mean Intensity vs Maximum Y Coordinate')
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'feature_vs_coordinates.png'))
    plt.close()

def preliminary_analysis(df, data, output_dir="proton_dataset", image_shape=(56, 30)):
    """Main analysis function that saves all outputs"""
    os.makedirs(output_dir, exist_ok=True)

    # Create a logger that writes to both console and file
    class DualLogger:
        def __init__(self, log_file):
            self.console = sys.stdout
            self.log_file = open(log_file, 'w')

        def write(self, message):
            self.console.write(message)
            self.log_file.write(message)

        def flush(self):
            self.console.flush()
            self.log_file.flush()

        def close(self):
            self.log_file.close()

    logger = DualLogger(os.path.join(output_dir, 'analysis_report.txt'))

    try:
        sys.stdout = logger

        print("=== Starting Analysis ===\n")
        print("=== Coordinate Distribution Analysis ===")
        stats = coordinate_distribution_analysis(df, output_dir, image_shape=image_shape)

        print("\n=== Visualizing Sample Images ===")
        sample_indices = visualize_sample_images(df, data, output_dir, image_shape=image_shape)

        print("\n=== Image-Coordinate Relationship Analysis ===")
        image_coordinate_relationship(df, data, sample_indices, output_dir)

        print("\n=== Analysis Complete ===")
    finally:
        sys.stdout = logger.console
        logger.close()

# Load the above created files for coordinates

`data_zp_path` should contain the path to the data pickle file containing the images from the proton calorimeter generated from `data_filtering.ipynb`.
`data_zn_path` should contain the path to the data pickle file containing the images from the neutron calorimeter generated from `data_filtering.ipynb`.
`coordinates_zp` should contain the path to the pickle file created above, containing the coordinates extracted from the proton calorimeter images.
`coordinates_zn` should contain the path to the pickle file created above, containing the coordinates extracted from the neutron calorimeter images.

## Analysis for ZP

In [4]:
# load the data
coordinates_zp = pd.read_pickle(r"data\data_coord_photonsum_proton_1_2312.pkl")
data_zp = pd.read_pickle(r"data\data_proton_photonsum_proton_1_2312.pkl")

In [5]:
output_dir = "zp_coordinates_analysis/"
preliminary_analysis(coordinates_zp, data_zp, output_dir)

=== Starting Analysis ===

=== Coordinate Distribution Analysis ===
Statistical summary:
        min   max       mean  median        std
max_x  0.0  55.0  12.214558     6.0  14.164982
max_y  0.0  29.0  13.605773    14.0   8.829163

=== Visualizing Sample Images ===

=== Image-Coordinate Relationship Analysis ===
Correlation between mean image intensity and max_x: -0.629
Correlation between mean image intensity and max_y: 0.207

=== Analysis Complete ===


## Analysis for ZN

In [4]:
# load the data
coordinates_zn = pd.read_pickle(r"data\data_coord_photonsum_neutron_1_3360.pkl")
data_zn = pd.read_pickle(r"data\data_neutron_photonsum_neutron_1_3360.pkl")

In [5]:
output_dir = "zn_coordinates_analysis/"
preliminary_analysis(coordinates_zn, data_zn, output_dir, image_shape=(44, 44))

=== Starting Analysis ===

=== Coordinate Distribution Analysis ===
Statistical summary:
        min   max       mean  median        std
max_x  0.0  43.0  22.250632    22.0  12.713540
max_y  0.0  43.0  21.830041    22.0  13.147766

=== Visualizing Sample Images ===

=== Image-Coordinate Relationship Analysis ===
Correlation between mean image intensity and max_x: -0.664
Correlation between mean image intensity and max_y: -0.372

=== Analysis Complete ===
